In [120]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

In [162]:
# Reading In Weather CSV file

weather_df = pd.read_csv('weather.csv')

# weather_df

## Humidity Heatmap

In [57]:
# GMap Configure 

gmaps.configure(g_key)

In [58]:
# Setting Variables 

locations = weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity"].astype(float)

In [59]:
# Creating Heat Map 

heatfig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=humidity.max(),
                                 point_radius = 3)

heatfig.add_layer(heat_layer)

heatfig

Figure(layout=FigureLayout(height='420px'))

## Ideal Weather Criteria

In [163]:
# Duplicate Of Weather Data Frame 

hotel_df = weather_df

# hotel_df 

In [164]:
# Ideal Weather Condition: Less Than 80 Degrees

lessthan80_df = hotel_df.loc[hotel_df["Max Temp"] <= 80]

# lessthan80_df

In [165]:
# Ideal Weather Condition: Greater Than 70 Degrees

higherthan70_df = lessthan80_df.loc[lessthan80_df["Max Temp"] >= 70]

#higherthan70_df

In [166]:
# Ideal Weather Condition: Wind Speed Less Than 10 mph 

lessthan10mph_df = higherthan70_df.loc[higherthan70_df["Wind Speed"] < 10]

# lessthan10mph_df

In [167]:
# Ideal Weather Condition: No clouds

zeroclouds_df = lessthan10mph_df.loc[lessthan10mph_df["Cloudiness"]== 0]

# zeroclouds_df

In [71]:
# Ideal Locations DataFrame 

perfect_vacation_df = zeroclouds_df

In [104]:
# Pick Nine Random Ideal Locations 
# https://www.geeksforgeeks.org/how-to-randomly-select-rows-from-pandas-dataframe/

nine_random_vacation_hotels_df = perfect_vacation_df.sample(n = 9)
nine_random_vacation_hotels_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
62,goderich,43.7501,-81.7165,71.15,43,0,5.66,CA,1616163703
160,korla,41.7597,86.1469,77.19,69,0,0.72,CN,1616163616
450,fort-shevchenko,44.5086,50.2630,78.34,81,0,5.59,KZ,1616163896
403,parainen,60.3067,22.3010,71.48,34,0,2.57,FI,1616163873
515,nemuro,43.3236,145.5750,71.27,77,0,6.53,JP,1616163930
141,hami,42.8000,93.4500,74.92,27,0,1.18,CN,1616163743
330,urumqi,43.8010,87.6005,72.15,74,0,2.00,CN,1616163605
474,orebro,59.2741,15.2066,77.15,24,0,2.57,SE,1616163908
177,esterhazy,50.6500,-102.0843,72.15,86,0,3.09,CA,1616163760


## Hotel Map

In [169]:
# Adding Hotel Name Column

nine_random_vacation_hotels_df["Hotel Name"] = ""

# nine_random_vacation_hotels_df

In [158]:
# Search URL And Params

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
          "radius":5000,
          "type":"hotel",
          "keyword":"hotel",
          "key":g_key
         }

for index, row in nine_random_vacation_hotels_df.iterrows():
    
    lat_int = row['Lat']
    lng_int = row['Lng']
    
    # Converting Variables To String
    # https://pythonprinciples.com/blog/converting-integer-to-string-in-python/
    
    lat_string = str(lat_int)
    lng_string = str(lng_int)
    
    params['location'] = f"{lat_string},{lng_string}"
    
    #Getting Responses
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    #Inputting Hotel Names To DF
    
    try: 
        hotel_name = results[1]['name']
        nine_random_vacation_hotels_df.loc[index, 'Hotel Name'] = hotel_name
        print(f"Hotels Found! {hotel_name}")
    except:
        nine_random_vacation_hotels_df.loc[index, 'Hotel Name'] = 'N/A'
        print("Hotel Not Found!")

Hotels Found! Comfort Inn & Suites
Hotels Found! the puju
Hotels Found! Eles Hotel
Hotel Not Found!
Hotels Found! TomaYa
Hotels Found! 哈密环球大酒店
Hotels Found! JinJiang International Hotel Urumqi
Hotels Found! Elite Stora Hotellet Örebro
Hotels Found! Western Star Inn & Suites - Esterhazy


In [170]:
# Making Sure That Hotel Names Are In DataFrame 
# nine_random_vacation_hotels_df

In [160]:
# Using The Template Add The Hotel Marks To The Heatmap

info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store The DataFrame Row
# NOTE: be sure to update with your DataFrame name

hotel_info = [info_box_template.format(**row) for index, row in nine_random_vacation_hotels_df.iterrows()]
locations = nine_random_vacation_hotels_df[["Lat", "Lng"]]

In [161]:
# Adding Marker Locations To DataFrame 

markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)
heatfig.add_layer(markers)
heatfig

Figure(layout=FigureLayout(height='420px'))